Reference
* https://www.kaggle.com/code/yasufuminakama/pppm-deberta-v3-large-baseline-w-w-b-train
* https://www.kaggle.com/code/debarshichanda/pytorch-feedback-deberta-v3-baseline

# Library

In [1]:
import os
import gc
import math
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns #可视化
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm #进度条

import torch
import torch.nn as nn #神经网络接口
import torch.nn.functional as F #nn库的所有函数
import torch.optim as optim #优化算法库
from torch.optim import lr_scheduler #调整学习率
from torch.utils.data import DataLoader, Dataset #Dataset 抽象类；DataLoader 是接口

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

from transformers import AutoModel, AutoConfig, AutoTokenizer, AdamW, DataCollatorWithPadding
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Directly Setting

In [2]:
INPUT_DIR = '../input/feedback-price-datasets-with-essay-text/'
OUTPUT_DIR = './baseline/'

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# CFG

In [3]:
class CFG:
    wandb = False
    apex = True
    model = 'microsoft/deberta-v3-base'
    seed = 42
    n_splits = 4
    max_len = 512
    dropout = 0.2
    target_size=3
    n_accumulate=1
    print_freq = 100
    min_lr=1e-6
    scheduler = 'cosine'
    batch_size = 8
    num_workers = 3
    lr = 2e-5
    weigth_decay = 0.01
    epochs = 4
    n_fold = 4
    trn_fold = [0, 1, 2, 3]
    train = True 
    num_warmup_steps = 0
    num_cycles=0.5
    debug = True
    debug_ver2 = False

if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0, 1]
    CFG.print_freq = 10
    
if CFG.debug_ver2:
    CFG.epochs = 1
    CFG.trn_fold = [0, 1]

# Utils

In [4]:
def criterion(outputs, labels):
    return nn.CrossEntropyLoss()(outputs, labels)
"""
def get_score(outputs, labels):
    return log_loss(labels, outputs)
"""
def get_score(outputs, labels):
    outputs = F.softmax(torch.tensor(outputs)).numpy()
    return log_loss(labels, outputs)

def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=CFG.seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# DataLoading

In [5]:
train = pd.read_csv(INPUT_DIR+'train_all.csv')
test = pd.read_csv(INPUT_DIR+'test_all.csv')
display(train.head())
print(train.shape)
display(test.head())
print(test.shape)

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,essay_text
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."


(36765, 6)


,discourse_id,essay_id,discourse_text,discourse_type,essay_text
0,a261b6e14276,D72CB1C11673,Making choices in life can be very difficult. ...,Lead,Making choices in life can be very difficult. ...
1,5a88900e7dc1,D72CB1C11673,Seeking multiple opinions can help a person ma...,Position,Making choices in life can be very difficult. ...
2,9790d835736b,D72CB1C11673,it can decrease stress levels,Claim,Making choices in life can be very difficult. ...
3,75ce6d68b67b,D72CB1C11673,a great chance to learn something new,Claim,Making choices in life can be very difficult. ...
4,93578d946723,D72CB1C11673,can be very helpful and beneficial.,Claim,Making choices in life can be very difficult. ...


(10, 5)


# CV splits

In [6]:
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
train['fold'] = -1
train['label'] = train['discourse_effectiveness'].map({'Ineffective':0, 'Adequate':1, 'Effective':2})
for i, (_, val_) in enumerate(skf.split(train, train['label'])):
    train.loc[val_, 'fold'] = int(i)
train.fold.value_counts()

0    9192
1    9191
3    9191
2    9191
Name: fold, dtype: int64

In [7]:
if CFG.debug:
    display(train.groupby('fold').size())
    train = train.sample(n=1000, random_state=0).reset_index(drop=True)
    display(train.groupby('fold').size())

fold
0    9192
1    9191
2    9191
3    9191
dtype: int64

fold
0    263
1    253
2    252
3    232
dtype: int64

# tokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
CFG.tokenizer = tokenizer

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Dataset

In [9]:
train['text'] = train['discourse_text'] + '[SEP]' + train['essay_text']

class FeedBackDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = CFG.max_len
        self.text = df['text'].values
        self.tokenizer = CFG.tokenizer
        self.targets = df['label'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        text = self.text[index]
        inputs = tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length = self.max_len
        )
        return {
            'input_ids':inputs['input_ids'],
            'attention_mask':inputs['attention_mask'],
            'target':self.targets[index]
            }

In [10]:
collate_fn = DataCollatorWithPadding(tokenizer=CFG.tokenizer)

# Model

In [11]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

In [12]:
class FeedBackModel(nn.Module):
    def __init__(self, model_name):
        super(FeedBackModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.config = AutoConfig.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.pooler = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, CFG.target_size)
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids, 
                         attention_mask=mask,
                         output_hidden_states=False)
        out = self.pooler(out.last_hidden_state, mask)
        out = self.drop(out)
        outputs = self.fc(out)
        return outputs

# HelperFunction

In [13]:
def asMinutes(s):
    m = math.floor(s/60)
    s -= m * 60
    return "%dm %ds" % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))

def get_scheduler(cfg, optimizer, num_train_steps):
    if cfg.scheduler == 'linear':
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
        )
    elif cfg.scheduler == 'cosine':
        scheduler = get_cosine_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
        )
    return scheduler

def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()

    dataset_size = 0
    running_loss = 0

    start = end = time.time()

    for step, data in enumerate(dataloader):
        ids = data['input_ids'].to(device, dtype=torch.long)
        mask = data['attention_mask'].to(device, dtype=torch.long)
        targets = data['target'].to(device, dtype=torch.long)

        batch_size = ids.size(0)
        
        outputs = model(ids, mask)
        loss = criterion(outputs, targets)

        #accumulate
        loss = loss / CFG.n_accumulate 
        loss.backward()
        if (step +1) % CFG.n_accumulate == 0:
            optimizer.step()

            optimizer.zero_grad()
            if scheduler is not None:
                scheduler.step()
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        end = time.time()
        
        if step % CFG.print_freq == 0 or step == (len(dataloader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  .format(epoch+1, step, len(dataloader), 
                          remain=timeSince(start, float(step+1)/len(dataloader))))

    gc.collect()

    return epoch_loss


@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0

    start = end = time.time()
    pred = []

    for step, data in enumerate(dataloader):
        ids = data['input_ids'].to(device, dtype=torch.long)
        mask = data['attention_mask'].to(device, dtype=torch.long)
        targets = data['target'].to(device, dtype=torch.long)

        batch_size = ids.size(0)
        outputs = model(ids, mask)
        loss = criterion(outputs, targets)
        pred.append(outputs.to('cpu').numpy())

        running_loss += (loss.item()* batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        end = time.time()

        if step % CFG.print_freq == 0 or step == (len(dataloader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  .format(step, len(dataloader),
                          remain=timeSince(start, float(step+1)/len(dataloader))))
            
    pred = np.concatenate(pred)
            
    return epoch_loss, pred

In [14]:
def train_loop(fold):
    #wandb.watch(model, log_freq=100)

    LOGGER.info(f'-------------fold:{fold} training-------------')

    train_data = train[train.fold != fold].reset_index(drop=True)
    valid_data = train[train.fold == fold].reset_index(drop=True)
    valid_labels = valid_data.label.values

    trainDataset = FeedBackDataset(train_data, CFG.tokenizer, CFG.max_len)
    validDataset = FeedBackDataset(valid_data, CFG.tokenizer, CFG.max_len)

    train_loader = DataLoader(trainDataset,
                              batch_size = CFG.batch_size,
                              shuffle=True,
                              collate_fn = collate_fn,
                              num_workers = CFG.num_workers,
                              pin_memory = True,
                              drop_last=True)
    
    valid_loader = DataLoader(validDataset,
                              batch_size = CFG.batch_size,
                              shuffle=False,
                              collate_fn = collate_fn,
                              num_workers = CFG.num_workers,
                              pin_memory = True,
                              drop_last=False)
    
    model = FeedBackModel(CFG.model)
    torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weigth_decay)
    num_train_steps = int(len(train_data) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # loop
    best_score = 100

    for epoch in range(CFG.epochs):
        start_time = time.time()

        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, train_loader, device, epoch)
        valid_epoch_loss, pred = valid_one_epoch(model, valid_loader, device, epoch)

        score = get_score(pred, valid_labels)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {train_epoch_loss:.4f}  avg_val_loss: {valid_epoch_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')
        if CFG.wandb:
            wandb.log({f"[fold{fold}] epoch": epoch+1, 
                       f"[fold{fold}] avg_train_loss": train_epoch_loss, 
                       f"[fold{fold}] avg_val_loss": valid_epoch_loss,
                       f"[fold{fold}] score": score})
            
        if score < best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': pred},
                        OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")
            
    predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_data['pred_0'] = predictions[:, 0]
    valid_data['pred_1'] = predictions[:, 1]
    valid_data['pred_2'] = predictions[:, 2]


    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_data

# Train

In [15]:
if __name__ == '__main__':
    
    def get_result(oof_df):
        labels = oof_df['label'].values
        preds = oof_df[['pred_0', 'pred_1', 'pred_2']].values.tolist()
        score = get_score(preds, labels)
        LOGGER.info(f'Score: {score:<.4f}')
    
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        oof_df.to_pickle(OUTPUT_DIR+'oof_df.pkl')
        oof_df.to_csv(OUTPUT_DIR+f'oof_df.csv', index=False)
        
    if CFG.wandb:
        wandb.finish()

-------------fold:0 training-------------


Downloading:   0%|          | 0.00/354M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: [1][0/92] Elapsed 0m 2s (remain 3m 15s) 
Epoch: [1][10/92] Elapsed 0m 8s (remain 1m 4s) 
Epoch: [1][20/92] Elapsed 0m 15s (remain 0m 51s) 
Epoch: [1][30/92] Elapsed 0m 21s (remain 0m 43s) 
Epoch: [1][40/92] Elapsed 0m 28s (remain 0m 35s) 
Epoch: [1][50/92] Elapsed 0m 35s (remain 0m 28s) 
Epoch: [1][60/92] Elapsed 0m 41s (remain 0m 21s) 
Epoch: [1][70/92] Elapsed 0m 48s (remain 0m 14s) 
Epoch: [1][80/92] Elapsed 0m 55s (remain 0m 7s) 
Epoch: [1][90/92] Elapsed 1m 1s (remain 0m 0s) 
Epoch: [1][91/92] Elapsed 1m 2s (remain 0m 0s) 
EVAL: [0/33] Elapsed 0m 0s (remain 0m 15s) 
EVAL: [10/33] Elapsed 0m 2s (remain 0m 5s) 
EVAL: [20/33] Elapsed 0m 4s (remain 0m 2s) 
EVAL: [30/33] Elapsed 0m 7s (remain 0m 0s) 


Epoch 1 - avg_train_loss: 0.9104  avg_val_loss: 0.8063  time: 70s
Epoch 1 - Score: 0.8063
Epoch 1 - Save Best Score: 0.8063 Model


...

========== fold: 0 result ==========
Score: 0.8000
-------------fold:1 training-------------
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSe

Epoch: [1][0/93] Elapsed 0m 1s (remain 1m 38s) 
Epoch: [1][10/93] Elapsed 0m 7s (remain 0m 57s) 
Epoch: [1][20/93] Elapsed 0m 14s (remain 0m 49s) 
Epoch: [1][30/93] Elapsed 0m 20s (remain 0m 41s) 
Epoch: [1][40/93] Elapsed 0m 27s (remain 0m 34s) 
Epoch: [1][50/93] Elapsed 0m 34s (remain 0m 28s) 
Epoch: [1][60/93] Elapsed 0m 40s (remain 0m 21s) 
Epoch: [1][70/93] Elapsed 0m 47s (remain 0m 14s) 
Epoch: [1][80/93] Elapsed 0m 53s (remain 0m 7s) 
Epoch: [1][90/93] Elapsed 1m 0s (remain 0m 1s) 
Epoch: [1][92/93] Elapsed 1m 1s (remain 0m 0s) 
EVAL: [0/32] Elapsed 0m 0s (remain 0m 15s) 
EVAL: [10/32] Elapsed 0m 2s (remain 0m 5s) 
EVAL: [20/32] Elapsed 0m 5s (remain 0m 2s) 
EVAL: [30/32] Elapsed 0m 7s (remain 0m 0s) 
EVAL: [31/32] Elapsed 0m 7s (remain 0m 0s) 


Epoch 1 - avg_train_loss: 0.9002  avg_val_loss: 0.8615  time: 70s
Epoch 1 - Score: 0.8615
Epoch 1 - Save Best Score: 0.8615 Model


Epoch: [2][0/93] Elapsed 0m 0s (remain 1m 23s) 
Epoch: [2][10/93] Elapsed 0m 7s (remain 0m 56s) 
Epoch: [2][20/93] Elapsed 0m 14s (remain 0m 48s) 
Epoch: [2][30/93] Elapsed 0m 20s (remain 0m 41s) 
Epoch: [2][40/93] Elapsed 0m 27s (remain 0m 34s) 
Epoch: [2][50/93] Elapsed 0m 34s (remain 0m 28s) 
Epoch: [2][60/93] Elapsed 0m 40s (remain 0m 21s) 
Epoch: [2][70/93] Elapsed 0m 47s (remain 0m 14s) 
Epoch: [2][80/93] Elapsed 0m 53s (remain 0m 7s) 
Epoch: [2][90/93] Elapsed 1m 0s (remain 0m 1s) 
Epoch: [2][92/93] Elapsed 1m 1s (remain 0m 0s) 
EVAL: [0/32] Elapsed 0m 0s (remain 0m 14s) 
EVAL: [10/32] Elapsed 0m 2s (remain 0m 5s) 
EVAL: [20/32] Elapsed 0m 4s (remain 0m 2s) 
EVAL: [30/32] Elapsed 0m 7s (remain 0m 0s) 
EVAL: [31/32] Elapsed 0m 7s (remain 0m 0s) 


Epoch 2 - avg_train_loss: 0.7686  avg_val_loss: 0.8154  time: 69s
Epoch 2 - Score: 0.8154
Epoch 2 - Save Best Score: 0.8154 Model
========== fold: 1 result ==========
Score: 0.8154
========== CV ==========
Score: 0.8076


In [16]:
A = pd.read_csv(OUTPUT_DIR+'oof_df.csv')
A.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,essay_text,fold,label,text,pred_0,pred_1,pred_2
0,5922babc4c05,746BC0AD4761,they need to keep the most popular one by elec...,Position,Ineffective,"Well to start with ,they need to keep the most...",0,0,they need to keep the most popular one by elec...,0.593548,1.185210,-1.772284
1,e677fc9a0c30,53E064890ABC,Without this policy students who say they want...,Evidence,Effective,The principal should create a rule for each st...,0,2,Without this policy students who say they want...,-1.436286,0.835475,2.152925
2,12c9ae107837,C133EB596A40,Many people might still think that mars landfo...,Concluding Statement,Ineffective,The face on Mars was actually landform on mars...,0,0,Many people might still think that mars landfo...,0.309734,1.282699,-1.861232
3,2abfe669c007,6621856784A9,In today's world our parents have many fears f...,Evidence,Adequate,"Dear principle,\n\nI think you should allow th...",0,1,In today's world our parents have many fears f...,0.282619,1.013161,-1.373789
4,19c85aa40fae,AA0FF7AA95C3,"In conculsion, ""It's offical: The electoral co...",Concluding Statement,Effective,"Dear State Senator, 1/23/15\n\nI have a very ...",0,2,"In conculsion, ""It's offical: The electoral co...",-0.569896,0.914341,-0.778971
